In [1]:
import sqlite3
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine
from sqlalchemy.pool import StaticPool

def get_db_engine(db_path):
    """로컬 SQLite DB 파일과 연결된 엔진을 생성합니다."""
    try:
        # SQLite DB 파일과 연결
        connection = sqlite3.connect(db_path, check_same_thread=False)
        
        # SQLAlchemy 엔진 생성
        engine = create_engine(
            f"sqlite:///{db_path}",
            poolclass=StaticPool,
            connect_args={"check_same_thread": False}
        )
        
        return engine
    
    except Exception as e:
        print(f"데이터베이스 연결 중 오류 발생: {str(e)}")
        return None

# DB 파일 경로 지정
db_path = './data/real_estate_transactions.db'

# 엔진 생성
engine = get_db_engine(db_path)

# LangChain SQLDatabase 객체 생성
db = SQLDatabase(engine)

In [7]:
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [9]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [10]:
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,
)

In [34]:
from langchain.prompts import SystemMessagePromptTemplate, MessagesPlaceholder
from langchain.prompts import ChatPromptTemplate

db_info = db.get_table_info()
info_tool = InfoSQLDatabaseTool(db=db)
schema_info = info_tool.run("apartment_rent")

# 사용자 정의 시스템 프롬프트 템플릿 생성
custom_system_prompt = """
당신은 SQL 전문가입니다. 
데이터베이스는 {dialect} 방언을 사용합니다.

다음 규칙을 따라주세요:
1. 한국어로 응답해주세요.
2. SQL 쿼리를 작성할 때는 명확하고 효율적이어야 합니다.
3. 결과는 최대 {top_k}개까지만 보여주세요.
4. 금액에 관련된 쿼리를 작성할 때는 쉼표(,)를 제거하고 숫자로 변환해야 합니다.
5. 에러가 발생하면 원인을 설명하고 수정된 쿼리를 제시해주세요.

예를 들어서 사용자 응답이 '서울시 강남구 1억 이하 전세를 추천해줘'라고 온다면
SELECT 시군구, 단지명, "보증금(만원)", 전용면적(㎡), 층 
FROM apartment_rent 
WHERE 시군구 LIKE '%강남구%' 
AND CAST(REPLACE("보증금(만원)", ',', '') AS INTEGER) < 10000 
AND 전월세구분 = '전세' 
LIMIT 5;
이런식으로 쿼리를 짜서 날려줍니다.

사용 가능한 테이블 스키마:
{schema_info}
"""

In [43]:
from langchain import hub
from langgraph.prebuilt import create_react_agent

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")
assert len(prompt_template.messages) == 1
print(prompt_template.input_variables)
system_message = prompt_template.format(dialect="SQLite", top_k=5)
agent_executor = create_react_agent(
    llm, toolkit.get_tools(), state_modifier=system_message
)

c:\Users\USER\anaconda3\envs\nlp\lib\site-packages\langsmith\client.py:354: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


['dialect', 'top_k']


In [35]:
prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(custom_system_prompt),
    MessagesPlaceholder(variable_name="messages"),
])

In [36]:
system_message = prompt_template.format(dialect="SQLite", top_k=5, messages=[])

In [37]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(
    llm, toolkit.get_tools(), state_modifier=system_message
)

In [44]:
example_query = "강남구에서 전세 보증금이 가장 저렴한 아파트의 가격은 얼마인가요?"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    config={"recursion_limit": 50},  # 재귀 제한을 50으로 증가
    stream_mode="values",
)

for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

강남구에서 전세 보증금이 가장 저렴한 아파트의 가격은 얼마인가요?
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_zKKNQVlBNLefUVJ0NpBagRhr)
 Call ID: call_zKKNQVlBNLefUVJ0NpBagRhr
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

apartment_rent, apartment_sale, commercial_sale, land_sale, officetel_rent, officetel_sale, single_house_rent, single_house_sale, subscription_rights_sale, townhouse_rent, townhouse_sale
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_1AbxDjDzGuASNVVHSWwO1bp7)
 Call ID: call_1AbxDjDzGuASNVVHSWwO1bp7
  Args:
    table_names: apartment_rent
  sql_db_schema (call_kenMj9JTkhTuGBTmi1RsPoYe)
 Call ID: call_kenMj9JTkhTuGBTmi1RsPoYe
  Args:
    table_names: apartment_sale
================================= Tool Messa

In [32]:
# 테이블 스키마 확인
info_tool = InfoSQLDatabaseTool(db=db)
print("Schema:", info_tool.run("apartment_rent"))

# 테이블의 실제 데이터 샘플 확인
query_tool = QuerySQLDataBaseTool(db=db)
sample_query = """
SELECT * FROM apartment_rent LIMIT 1;
"""
print("\n샘플 데이터:", query_tool.run(sample_query))

Schema: 
CREATE TABLE apartment_rent (
	"NO" INTEGER, 
	"시군구" TEXT, 
	"번지" TEXT, 
	"본번" INTEGER, 
	"부번" INTEGER, 
	"단지명" TEXT, 
	"전월세구분" TEXT, 
	"전용면적(㎡)" REAL, 
	"계약년월" INTEGER, 
	"계약일" INTEGER, 
	"보증금(만원)" TEXT, 
	"월세금(만원)" TEXT, 
	"층" INTEGER, 
	"건축년도" INTEGER, 
	"도로명" TEXT, 
	"계약기간" TEXT, 
	"계약구분" TEXT, 
	"갱신요구권 사용" TEXT, 
	"종전계약 보증금(만원)" TEXT, 
	"종전계약 월세(만원)" TEXT, 
	"주택유형" TEXT
)

/*
3 rows from apartment_rent table:
NO	시군구	번지	본번	부번	단지명	전월세구분	전용면적(㎡)	계약년월	계약일	보증금(만원)	월세금(만원)	층	건축년도	도로명	계약기간	계약구분	갱신요구권 사용	종전계약 보증금(만원)	종전계약 월세(만원)	주택유형
1	서울특별시 성동구 옥수동	220-1	220	1	한남하이츠	전세	177.12	202412	12	75,000	0	5	1982	독서당로 156	202412~202612	신규	-	None	None	아파트
2	서울특별시 관악구 신림동	1730	1730	0	신림푸르지오	월세	84.79	202412	12	25,000	95	13	2005	남부순환로 1430	202412~202612	갱신	-	35,000	40	아파트
3	서울특별시 도봉구 창동	27	27	0	주공19단지(창동리버타운)	월세	68.86	202412	12	15,000	90	6	1988	노해로70길 19	202501~202701	신규	-	None	None	아파트
*/

샘플 데이터: [(1, '서울특별시 성동구 옥수동', '220-1', 220, 1, '한남하이츠', '전세', 177.12, 202412, 12, '75,000', '0', 5, 1982,

In [20]:
query_tool = QuerySQLDataBaseTool(db=db)
test_query = """
SELECT DISTINCT 시군구, 전월세구분, COUNT(*) as count
FROM apartment_rent 
WHERE 시군구 LIKE '%강남%'
GROUP BY 시군구, 전월세구분;
"""

In [21]:
print("\n강남구 데이터 현황:")
print(query_tool.run(test_query))


강남구 데이터 현황:
[('서울특별시 강남구 개포동', '월세', 75), ('서울특별시 강남구 개포동', '전세', 104), ('서울특별시 강남구 논현동', '월세', 15), ('서울특별시 강남구 논현동', '전세', 16), ('서울특별시 강남구 대치동', '월세', 59), ('서울특별시 강남구 대치동', '전세', 62), ('서울특별시 강남구 도곡동', '월세', 32), ('서울특별시 강남구 도곡동', '전세', 58), ('서울특별시 강남구 삼성동', '월세', 34), ('서울특별시 강남구 삼성동', '전세', 16), ('서울특별시 강남구 세곡동', '월세', 9), ('서울특별시 강남구 세곡동', '전세', 15), ('서울특별시 강남구 수서동', '월세', 11), ('서울특별시 강남구 수서동', '전세', 42), ('서울특별시 강남구 신사동', '월세', 4), ('서울특별시 강남구 압구정동', '월세', 15), ('서울특별시 강남구 압구정동', '전세', 26), ('서울특별시 강남구 역삼동', '월세', 50), ('서울특별시 강남구 역삼동', '전세', 49), ('서울특별시 강남구 율현동', '월세', 1), ('서울특별시 강남구 율현동', '전세', 3), ('서울특별시 강남구 일원동', '월세', 30), ('서울특별시 강남구 일원동', '전세', 45), ('서울특별시 강남구 자곡동', '월세', 19), ('서울특별시 강남구 자곡동', '전세', 26), ('서울특별시 강남구 청담동', '월세', 8), ('서울특별시 강남구 청담동', '전세', 13)]


In [22]:
detail_query = """
SELECT 시군구, 단지명, "보증금(만원)" as deposit, 전월세구분
FROM apartment_rent 
WHERE 시군구 LIKE '%강남%'
AND 전월세구분 = '전세'
ORDER BY CAST(REPLACE("보증금(만원)", ',', '') AS INTEGER) ASC
LIMIT 5;
"""

In [23]:

print("\n강남구 전세 데이터 상세:")
print(query_tool.run(detail_query))


강남구 전세 데이터 상세:
[('서울특별시 강남구 수서동', '까치마을', '11,000', '전세'), ('서울특별시 강남구 도곡동', '중명하니빌', '18,450', '전세'), ('서울특별시 강남구 세곡동', '세곡리엔파크(3단지)', '19,989', '전세'), ('서울특별시 강남구 세곡동', '세곡리엔파크(3단지)', '19,989', '전세'), ('서울특별시 강남구 논현동', '한양수자인어반게이트', '20,000', '전세')]


In [24]:
import pandas as pd

df = pd.read_csv('./data/processed_data.csv')
df


,id,district,주택유형,도로조건,area_size,대지면적(㎡),transaction_date,계약일,거래금액,건축년도,...,floor,분양권/입주권,유형,용도지역,건축물주용도,지분구분,동,등기일자,건물명,대지권면적(㎡)
0,단독다가구(매매)_000000,서울특별시 도봉구 쌍문동,단독,8m미만,65.72,89.9,2024-12,9,37800.0,1970.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,단독다가구(매매)_000001,서울특별시 성북구 정릉동,다가구,8m미만,293.08,133.0,2024-12,7,71000.0,1994.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,단독다가구(매매)_000002,서울특별시 마포구 상수동,단독,8m미만,25.83,53.0,2024-12,7,51000.0,1985.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,단독다가구(매매)_000003,서울특별시 동대문구 휘경동,다가구,8m미만,162.18,110.0,2024-12,7,139448.0,1992.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,단독다가구(매매)_000004,서울특별시 광진구 능동,다가구,8m미만,245.15,163.6,2024-12,6,196000.0,2012.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29389,오피스텔(전월세)_003463,서울특별시 종로구 숭인동,NaN,NaN,15.47,NaN,2024-11,14,NaN,2020.0,...,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29390,오피스텔(전월세)_003464,서울특별시 구로구 구로동,NaN,NaN,17.81,NaN,2024-11,14,NaN,2013.0,...,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29391,오피스텔(전월세)_003465,서울특별시 구로구 구로동,NaN,NaN,19.61,NaN,2024-11,14,NaN,2012.0,...,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29392,오피스텔(전월세)_003466,서울특별시 구로구 구로동,NaN,NaN,20.30,NaN,2024-11,14,NaN,2016.0,...,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
